# Trabalho 02 - Extração de Atributos e k-NN com k-fold   

## Setup inicial

### Importando bibliotecas:


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import scipy.io as io
import scipy.stats as stats
from IPython.display import Audio
from collections import Counter
from typing import Tuple

### Criando a base de dados

In [ ]:
dataFolder = "./Acordes"
files = os.listdir(dataFolder)

columns = ["file", "mean", "std", "skewness", "kurtosis", "class"]

chords = pd.DataFrame(
    data= [],
    columns = columns,
    index= range(len(files))
)

for index, file in enumerate(files):
    
    className = file.split("_").pop(0)
    className = className.lower()

    sampleRate, audio = io.wavfile.read(f"{dataFolder}/{file}")
    
    mean = np.mean(audio)
    std = np.std(audio)
    skewness = stats.skew(audio)
    kurtosis = stats.kurtosis(audio)

    chords.iloc[index] = [file, mean, std, skewness, kurtosis, className]

chords.shape

(100, 6)

- Visualizando o início do banco de dados:

In [4]:
chords.head(10)

,file,mean,std,skewness,kurtosis,class
0,Major_0.wav,2.05809,4783.324474,-0.160373,2.695587,major
1,Major_1.wav,1368.288385,4145.706046,0.346228,4.103374,major
2,Major_10.wav,1.335756,4761.082845,-0.265601,1.767541,major
3,Major_11.wav,1130.052298,3596.918809,0.476877,4.769523,major
4,Major_12.wav,1.946506,4806.190296,-0.209936,1.644306,major
5,Major_13.wav,1338.479344,3778.07876,0.03611,6.720824,major
6,Major_14.wav,2.711101,4004.661113,-0.185869,1.957693,major
7,Major_15.wav,1216.795535,3679.785739,0.404273,10.945738,major
8,Major_16.wav,2.77809,4048.767658,-0.150399,2.087869,major
9,Major_17.wav,1204.970441,3171.591704,0.343243,12.392234,major


- Função para facilitar a leitura do arquivo de audio:

In [5]:
#Função para ler o arquivo de audio
def readAudio(audioFile:str)->Tuple[int, np.array]:
    return io.wavfile.read(f"{dataFolder}/{file}")

## Extraindo atributos

Os atributos mais óbivios para reconhecer notas musicais são aqueles que estão intimamente relacionados com a frequêcia, já que as notas musicais possuem frequências específicas. Minha principal aposta de atributo é na razão da distância da frequencia entre a terça e a tónica e a distância entre a quinta e a tônica:
$\frac{F_t}{denominador}$

## Criando o modelo do K-NN

In [ ]:
def euclideanDistance(a: np.array, b: np.array) -> float:
    return np.sqrt(np.sum((a - b) ** 2))


class KNN():
    """Classificador KNN"""
    
    def __init__(self, k: int=3, dataset: pd.DataFrame=chords):
        self.k = k
        self.data = dataset.copy()

    def fit(self, features, labels):
        self.X_train = np.array(features)
        self.y_train = np.array(labels)

    def predict(self, features:np.array) -> str:

        """Prediz a classe de um conjunto de features

            > [INPUT]: Conjunto de features \n
            > [OUTPUT]: Classe prevista"""	
        
        X = np.array(features)
        return [self._predict(x) for x in X]
    
    def _predict(self, x):
        distances = [euclideanDistance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_labels).most_common(1)
        return most_common[0][0]

## Criando o modelo do K-Fold


In [ ]:
class KFold:
    """Classificador K-Fold de validação cruzada"""

    def __init__(self, n_splits: int = 10, shuffle: bool = True, random_seed: int = None):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_seed = random_seed

    def split(self, features: np.array, labels=np.array)-> Tuple[np.ndarray, np.ndarray]:
        
        """"Divide os dados em n partes para validação cruzada
            > [INPUT]: Array de features e labels\n
            > [OUTPUT]: Tupla de arrays com os índices de treino e teste para cada fold
        """

        n_samples = len(features)
        indices = np.arange(n_samples)

        if self.shuffle:
            rng = np.random.default_rng(self.random_seed)
            indices = rng.permutation(indices)
        
        fold_sizes = np.full(self.n_splits, n_samples // self.n_splits, dtype=int)
        fold_sizes[:n_samples % self.n_splits] += 1  # Distribui o resto


        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size

            #indices para teste e treino
            test_idx = indices[start:stop]
            train_idx = np.concatenate([indices[:start], indices[stop:]])

            current = stop
            yield train_idx, test_idx